In [ ]:
# hyungtai sim, 24.04.18.

vlib = c("tidyverse", "mashr", "ashr", "ggpubr", "data.table", "patchwork", "tidyseurat",
         "future.apply", "arrow", 'pheatmap', "Seurat", "hdWGCNA", "enrichR", "SeuratData",
         "Azimuth", "data.table", "presto", "Matrix")
lapply(vlib, require, character.only = TRUE, quietly = TRUE) |> suppressMessages()

base.dir="/path/to/data"
chr_wd = paste0(base.dir, "202404-sceQTLv7/")
setwd(chr_wd)

saveRDS.gz <- function(object,file,threads=parallel::detectCores()) {
  con <- pipe(paste0("pigz -p",threads," > ",file),"wb")
  saveRDS(object, file = con)
  close(con)
}

readRDS.gz <- function(file,threads=parallel::detectCores()) {
  con <- pipe(paste0("pigz -d -c -p",threads," ",file))
  object <- readRDS(file = con)
  close(con)
  return(object)
}


In [ ]:
## aggregation functions ####

aggregate_counts = function(count_mat, annot, method = 'sum', verbose = TRUE) {
  annot = annot %>% mutate(group = factor(group))
  cells = intersect(colnames(count_mat), annot$cell)
  count_mat = count_mat[,cells]
  annot = annot %>% filter(cell %in% cells)
  cell_dict = annot %>% {setNames(.$group, .$cell)} %>% droplevels
  cell_dict = cell_dict[cells]
  if (verbose) {
    print(table(cell_dict))
  }
  if (length(levels(cell_dict)) == 1) {
    count_mat_clust = count_mat %>% rowSums() %>% as.matrix %>% magrittr::set_colnames(levels(cell_dict))
    mean_mat_clust = count_mat_clust / as.numeric(table(cell_dict))
  } else {
    M = model.matrix(~ 0 + cell_dict) %>% magrittr::set_colnames(levels(cell_dict)) %>% Matrix(., sparse = TRUE)
    mtx_diag_cellcount = diag(1/table(cell_dict)) %>% Matrix(., sparse = TRUE)
    count_mat_clust = count_mat %*% M %>% Matrix(., sparse = TRUE)
    mean_mat_clust = count_mat_clust %*% mtx_diag_cellcount %>% Matrix(., sparse = TRUE)
  }
  if (method == 'sum') {
    return(count_mat_clust)
  } else if (method == 'mean') {
    return(mean_mat_clust)
  } else {
    cat("set the method in sum or mean. \n")
    stop()
  }
}



pull_barcode = function(metadata){
  results = metadata %>% 
  dplyr::select(CellID, sample_time) %>%
  dplyr::rename(cell =  CellID, group = sample_time) %>% 
  group_by(group) %>% add_count() %>% filter(n > 3) %>%
  select(-n)
  return(results)
}

create_pseudobulk = function(df_barcode, dgem_full){
  int_len = df_barcode %>% distinct(group) %>% pull(group) %>% length()
  dgem_filtered= dgem_full[,df_barcode$cell]
  pseudobulk_filtered_count = aggregate_counts(dgem_filtered > 0, df_barcode)
  # filter 2.
  ## we will count the number of cells with counts >= 1 each gene, from each sample-cluster
  ## we collect the expressed cell count and filter >=3, which indicates 3 or more cells express specific genes per sample-cluster combination
  pseudobulk_results = aggregate_counts(dgem_filtered, df_barcode, method = "mean")
  #pseudobulk_results[pseudobulk_filtered_count < 3] <- 0
  pseudobulk_filtered_genes = rowSums(pseudobulk_results > 0) %>% as.data.frame() %>%
    rownames_to_column("gene_name") %>% 
    filter(`.` >= 1.0*int_len) %>% pull(gene_name)
  colnames(pseudobulk_results) = colnames(pseudobulk_filtered_count)
  pseudobulk_results = pseudobulk_results[pseudobulk_filtered_genes,]
  return(pseudobulk_results)
}

plot_logexpression = function(pseudobulk_results) {
  p1 = pseudobulk_results %>% as.matrix() %>% as.data.frame() %>%
    rownames_to_column('gene_name') %>%
    pivot_longer(cols = c(2:ncol(.))) %>%
    group_by(name) %>%
    filter(value > 0) %>%
    ggplot(aes(x = name, y = value)) + 
    geom_boxplot() + 
    theme_pubr(x.text.angle = 90) +
    scale_y_log10()
  return(p1)
}

plot_pca = function(pseudobulk_results) {
  results_pca = vector(mode = "list", length = 3L)
  pca_obj = pseudobulk_results %>% prcomp()
  pca_obj_summary = pca_obj %>% 
    summary() %>% 
    .$importance %>% 
    t() %>% 
    as.data.frame()
  
  p1 = pca_obj$rotation %>% 
    as.data.frame() %>%
    ggplot(aes(x = PC1, y = PC2)) + 
    geom_point() +
    theme_pubr() +
    xlab(paste0("PC1 (",
                format(pca_obj_summary$`Proportion of Variance`[1]*100, digit=3),
                " % of Variance)")) +
    ylab(paste0("PC2 (", 
                format(pca_obj_summary$`Proportion of Variance`[2]*100, digit=3),
                " % of Variance)"))
  
  results_pca$pca = pca_obj
  results_pca$var_prop = pca_obj_summary
  results_pca$plot = p1
  print(p1)
  return(results_pca)
}

create_genotype = function(chr_path_si, chr_path_fam, df_sample_name, n_max = 73, chr_prefix, global_prefix){
  temp_prefix = "/home/rstudio/.local/temp/" # change if needed
  list_res = vector(mode = "list", length = 2L)
  names(list_res) = c("intersect", "genotype_prefix")
  df_si = read_delim(chr_path_si) %>%
    filter(IID %in% df_sample_name$sample) %>%
    select(IID, response, age, sex, histology)
  df_geno_all = read_delim(chr_path_fam, col_names = F)
  colnames(df_geno_all) = c("FID", "IID", "V3", "V4", "V5", "V6")
  intersect = left_join(df_si,df_geno_all, by = c("IID")) %>%
    mutate(sex = ifelse(sex == "M", "1", ifelse(sex == "F", "2", "0"))) 
  list_res$intersect = intersect
  ### check module if N = 73 (almost all) ####
  if (nrow(df_si) == n_max){
    # we don't need any precessing, since all samples are listed.
    #    subset_ids = intersect %>% 
    #      mutate(before_FID = IID, before_IID = IID) %>% 
    #      select(before_FID, before_IID) %>%
    #      distinct(before_FID, before_IID) %>% 
    #      write_delim(paste0(temp_prefix, "temp_plink.subset_ids.txt"), col_names = FALSE, delim = '\t')
    #    update_ids = intersect %>% 
    #      mutate(before_FID = 0, before_IID = IID, after_FID = IID, after_IID = IID) %>% 
    #      select(before_FID, before_IID, after_FID, after_IID) %>%
    #      distinct(before_FID, before_IID, .keep_all = TRUE) %>%
    #      write_delim(paste0(temp_prefix, "temp_plink.update_ids.txt"), col_names = FALSE, delim = '\t')
    #    update_sex = intersect %>% 
    #      mutate(FID = IID, IID = IID) %>% 
    #      select(FID, IID, sex) %>%
    #      distinct(FID, IID, .keep_all = TRUE) %>%
    #      write_delim(paste0(temp_prefix,"temp_plink.update_sex.txt"), col_names = FALSE, delim = '\t')
    #    system(paste0("analysis/00_02_01_plink_script.sh ", chr_prefix))
    list_res$genotype_prefix = paste0(global_prefix, "geno/all_plink_maf01_HWEe5_geno03_mind03")
  } else {
    subset_ids = intersect %>% 
      mutate(before_FID = IID, before_IID = IID) %>% 
      select(before_FID, before_IID) %>%
      distinct(before_FID, before_IID) %>% 
      write_delim(paste0(temp_prefix, "temp_plink.subset_ids.txt"), col_names = FALSE, delim = '\t')
    update_ids = intersect %>% 
      mutate(before_FID = 0, before_IID = IID, after_FID = IID, after_IID = IID) %>% 
      select(before_FID, before_IID, after_FID, after_IID) %>%
      distinct(before_FID, before_IID, .keep_all = TRUE) %>%
      write_delim(paste0(temp_prefix, "temp_plink.update_ids.txt"), col_names = FALSE, delim = '\t')
    update_sex = intersect %>% 
      mutate(FID = IID, IID = IID) %>% 
      select(FID, IID, sex) %>%
      distinct(FID, IID, .keep_all = TRUE) %>%
      write_delim(paste0(temp_prefix,"temp_plink.update_sex.txt"), col_names = FALSE, delim = '\t')
    system(paste0("analysis/00_02_01_plink_script.sh ", chr_prefix))
    list_res$genotype_prefix = paste0(global_prefix, "plink_subset/", chr_prefix, "_maf01_HWEe5_geno03_mind03")
  }
  return(list_res)
}


In [ ]:
# spare matrix reading and filtering ####

## read sparse matrix in SCTransformed raw count. ####

dgem = readRDS.gz("01_calling_eQTL/01_00_pseudobulk_pipeline/all_dgcmatrix_for_sceQTLv6a.rds")

## 1% or more expressed genes in cohort; generates character vector with genes.  ####
count_filtered_genes = rowSums(dgem > 0) %>%
  as.data.frame() %>%
  rownames_to_column("gene_name") %>%
  filter(`.` > 0.01 * dim(dgem)[2]) %>%
  pull(gene_name)
dgem = dgem[count_filtered_genes, ]

In [ ]:
# 2. iter.tools ####

## 2.0 일반적으로 Cluster level로 pipeline을 짜야 함. ####
### genotype :
### phenotype : cell id도 있으면 좋음 
### covariates :
#### genotype PCA, other MD : id만 필요
#### phenotype PCA : phenotype 전체가 필요함

## as a pipeline : phenotype 에서 나머지 만들기

global_prefix = "01_calling_eQTL/01_00_pseudobulk_pipeline/"
gtf = read_delim("gencode.v43.basic.tss_bed.txt")

chr_cluster_definition = "anno_l1"
chr_time_definition = "1st" # base for baseline, or 1st for treatment
chr_sampleid = "sample_time"
chr_path_sample_information = "00_sample_information_v8.txt"
# note: this is same as 220426_maf_0.1_hwe_1e-5_genomind_0.03_updateIID
chr_original_fam = "/path/to/data/202404-sceQTLv7/01_calling_eQTL/01_00_pseudobulk_pipeline/original/230628_chrALL_sample_ALLIO.fam"


## 2.1. define cluster, barcode df and sample list  ####

# @ 240123 modified. join-all and select

c_clusters = df_full_md_annotated %>% 
  group_by(!!! rlang::syms(c(chr_sampleid, chr_cluster_definition, "time"))) %>%
  summarise(n = n()) %>%
  filter(time == chr_time_definition, n >= 3) %>% ungroup() %>% select(-n) %>%
  group_by(get(chr_cluster_definition)) %>% add_count() %>% filter(n >= 0.85 * 73) %>%
  distinct(get(chr_cluster_definition)) %>% pull(.)


for (idx in c(1:length(c_clusters))){

  c_each_cluster = c_clusters[idx]
  list_path_res = vector(mode = "list", length = length(c_each_cluster))
  names(list_path_res) = c_each_cluster
  
  # check if needed
  chr_prefix = paste(chr_cluster_definition, chr_time_definition, c_each_cluster, sep = "_")
  chr_exprs_path = paste0(global_prefix,"exprs_matrix/", chr_prefix, ".bed")
  chr_raw_exprs_path = paste0(global_prefix,"exprs_matrix_raw/", chr_prefix, ".bed")
  chr_covs_path = paste0(global_prefix,"covariates/", chr_prefix, ".txt")
  chr_interaction_path = paste0(global_prefix,"interactions/", chr_prefix, ".txt")
  chr_qtlinput_path = paste0(global_prefix, "tensorqtl_input_l2/", chr_prefix, ".txt")
  
  df_groupinfo = df_full_md_annotated %>% 
    filter(sample %in% chr_all_samples) %>%
    #filter(is.na(CHIP_BEFORE_BINARY) == FALSE) %>% # as we don't need any interaction pipeline for NOW.
    filter(time %in% chr_time_definition, get(chr_cluster_definition) %in% c_each_cluster) %>%
    pull_barcode()
  
  df_sample_name = df_groupinfo %>% 
    separate(group, into = c("sample", "time"), sep = "-", remove = F) %>% ungroup() %>% distinct(sample)
  
  ## 2.2. sample sex annotation for genotype QC ####
  
  list_geno_res = create_genotype(
    chr_path_si = chr_path_sample_information,
    chr_path_fam = chr_original_fam, 
    df_sample_name = df_sample_name,
    n_max = 73
    chr_prefix = chr_prefix,
    global_prefix = global_prefix
  )
  
  ## 2.3. expression module ####
  
  pseudobulk_results = create_pseudobulk(df_groupinfo, dgem)
  pca_pseudobulk = plot_pca(pseudobulk_results)
  
  raw_exprs_df = pseudobulk_results %>% 
    as.matrix() %>% as.data.frame() %>% rownames_to_column("gene_id") %>%
    pivot_longer(cols = c(2:ncol(.))) %>%
    group_by(gene_id)
  
  raw_exprs_df %>% separate(name, into = c("name", "time"), sep = "-") %>% select(-time) %>%
    pivot_wider(names_from = name, values_from = value) %>%
    right_join(gtf, .) %>%
    select(1:4, df_sample_name$sample) %>%
    dplyr::rename(phenotype_id = gene_id) %>%
    distinct(`#chr`, start, end, .keep_all=TRUE) %>% add_count(phenotype_id) %>%
    filter(is.na(start) == FALSE, n ==1) %>% select(-n) %>%
    write_delim(chr_raw_exprs_path, delim = "\t", quote = "none")
  
  qnorm_df = raw_exprs_df %>%
    mutate(norm = qnorm((rank(value,na.last="keep")-0.5)/sum(!is.na(value)))) %>%
    select(-value) %>%
    separate(name, into = c("name", "time"), sep = "-") %>% select(-time) %>%
    pivot_wider(names_from = name, values_from = norm)
  
  right_join(gtf, qnorm_df) %>% select(1:4, df_sample_name$sample) %>%
    dplyr::rename(phenotype_id = gene_id) %>%
    distinct(`#chr`, start, end, .keep_all=TRUE) %>% add_count(phenotype_id) %>%
    filter(is.na(start) == FALSE, n ==1) %>% select(-n) %>%
    write_delim(chr_exprs_path, delim = "\t", quote = "none")
  
  # note : use miniconda to install tabix, bgzip and plink if needed
  system(paste0('~/miniconda3/bin/bgzip -f ', chr_exprs_path))
  system(paste0('~/miniconda3/bin/tabix -p bed -f ', chr_exprs_path, ".gz"))

  ## 2.4. Covariates module ####
  
  df_cov_base = fread(paste0(list_geno_res$genotype_prefix, ".fam")) %>% select(V2, V5) %>% 
    filter(V2 %in% df_sample_name$sample) %>% 
    mutate(V2 = factor(V2, levels = df_sample_name$sample)) %>%
    arrange(V2)%>%
    dplyr::rename(sex = V5, IID = V2)
  
  df_ePCA = pca_pseudobulk$pca$rotation %>% as.data.frame() %>% rownames_to_column("id_base") %>%
    separate(id_base, sep = "-", into = c("IID", "time")) %>%
    select(1,PC1:PC10) # 10 PCs
  
  df_gPCA =  fread(paste0(list_geno_res$genotype_prefix, ".eigenvec")) %>%
    select(1, V3:V5) %>% # 3 gPCs
    dplyr::rename(IID = V1)
  
  df_si = read_delim(chr_path_sample_information) %>% 
    select(IID, response, histology, age)
  
  df_file =  df_full_md_annotated %>%  
    distinct(sample_time, file, CHIP_BEFORE_BINARY) %>%
    separate(sample_time, into = c("name", "time"), sep = "-") %>% 
    filter(time == chr_time_definition, name %in% df_sample_name$sample) %>%
    select(-time) %>%
    dplyr::rename(IID = name) %>%
    mutate(CHIP_BEFORE_BINARY = ifelse(CHIP_BEFORE_BINARY== TRUE, 1, 0))
  
  df_cov = left_join(df_cov_base, df_ePCA) %>%
    left_join(., df_gPCA) %>%
    left_join(., df_si) %>%
    left_join(., df_file) %>%
    mutate(sex = sex -1) 
  
  df_cov %>% select(1:V5, age, file) %>%
    t() %>%
    write.table(chr_covs_path, quote = FALSE, col.names = F, sep = "\t") # gPC 3, ePC 10, sex 
  
  df_cov %>% select(1, response, histology) %>% t() %>%
    write.table(chr_interaction_path, quote = FALSE, col.names = F, sep = "\t")
  # sex is not used for interaction term. cannot be stratified for eqtl.
  # smoking is also not used for interaction term. cannot be stratified for eqtl.
  
  df_output_list = data.frame(
    geno_prefix = list_geno_res$genotype_prefix,
    pheno_path = paste0(chr_exprs_path, ".gz"),
    covs_path = chr_covs_path,
    interaction_path = chr_interaction_path,
    raw_pheno_path = chr_raw_exprs_path
  ) %>% pivot_longer(cols = 1:5)


  df_output_list %>% write_delim(chr_qtlinput_path, delim = "\t")
  
}
